# Assess Task Compute Time and Data Sizes
To get a better idea how they are each different and provide quantitative descriptions in the paper

In [1]:
from pathlib import Path
import pandas as pd
import json

Configuration

In [2]:
run_dir = Path('prod-runs/baseline_retrain-20/')

## Load in Each Type of Task
We'll need the runtimes and method sizes

In [3]:
tasks = []
for path in run_dir.glob('*-results.json'):
    records = pd.read_json(path, lines=True)
    records['total_size'] = records['message_sizes'].apply(lambda x: sum(x.values())) / 1e6
    tasks.append(records[['method', 'time_running', 'total_size', 'message_sizes']])
tasks = pd.concat(tasks)

## Group by Mean
Print out the statistics

In [4]:
tasks.groupby('method').describe(percentiles=[0.5])

time_running                                                  \
                      count        mean         std         min         50%   
method                                                                        
evaluate               16.0    0.223641    0.017172    0.201772    0.229842   
run_calculator        209.0  355.145035  291.140440    7.561529  304.595907   
run_sampling          175.0    2.159802    0.631202    1.164975    1.869510   
train                  88.0  260.607187    7.975323  248.836407  261.042843   

                            total_size                                  \
                        max      count       mean       std        min   
method                                                                   
evaluate           0.247943       16.0   3.213995  0.012332   3.202055   
run_calculator  1034.273956      211.0   0.020020  0.007531   0.005892   
run_sampling       3.974754      175.0   3.132809  0.016486   3.103596   
train            275.629800       88.0  20.966871  0.632828  19.986104   

                                      
                      50%        max  
method                                
evaluate         3.213995   3.225935  
run_calculator   0.021274   0.031910  
run_sampling     3.132072   3.190586  
train           20.965814  21.955550

## Get the size of the inference task
How many structures per chunk

In [5]:
with (run_dir / 'runparams.json').open() as fp:
    config = json.load(fp)

In [6]:
config['infer_chunk_size']

100